<a href="https://colab.research.google.com/github/haytham918/low-rank-expectile/blob/main/Low_Rank_Expectile_with_Simulated_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as plot
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [16]:
import numpy as np
import pandas as pd
from scipy.stats import norm

np.random.seed(500)

n_users = 50
n_items = 40

latent_dim = 2

true_r = norm.rvs(size=n_users)
true_c = norm.rvs(size=n_items)

true_u = norm.rvs(size=(n_users, latent_dim))
true_v = norm.rvs(size=(n_items, latent_dim))

sigma = 0.5

mu_X = np.outer(true_r, true_c) + np.dot(true_u, true_v.T)

error = sigma * norm.rvs(size=(n_users, n_items))

feature_matrix = mu_X + error

prob_miss = 0.2

missing = np.random.choice([True, False], size=(n_users, n_items), p=[prob_miss, 1- prob_miss])
feature_matrix[missing] = np.nan

nan_matrix = np.isnan(feature_matrix)
print(feature_matrix)

[[        nan  0.45365492 -1.14393308 ...         nan  0.10629064
   0.73474957]
 [-0.40606608 -0.41561065  1.28503233 ... -2.09206259 -0.17398039
  -1.52876877]
 [ 0.47972853  0.24906682  0.26750481 ... -1.43583817         nan
  -0.62707475]
 ...
 [-1.22378181  0.05043373         nan ...  0.98055671  0.94802885
  -0.38990545]
 [ 1.08027337         nan -0.36384272 ...  2.28791502 -4.96483045
   1.11112067]
 [ 0.15508994 -0.21717586         nan ...  0.75743742  0.71706363
   0.12058302]]


In [75]:
# Split the data into trainig/validation and exclude missing values
train_data, val_data, train_mask, val_mask = train_test_split(feature_matrix, nan_matrix, test_size=0.2, random_state=445)


# Create Tensors based on train/val data
train_tensor = torch.tensor(train_data, dtype=torch.float32)
print(train_tensor)
print(train_mask)
val_tensor = torch.tensor(val_data, dtype=torch.float32)


# Model definition
class LRModel(nn.Module):
  def __init__(self, number_users, number_times, rank):
    super().__init__()
    self.user_factors = nn.Embedding(number_users, rank)
    self.times_factors = nn.Embedding(number_times, rank)

    self.user_bias = nn.Embedding(number_users, 1)
    self.times_bias = nn.Embedding(number_times, 1)

    # Initializing the bias terms to zeros
    self.user_bias.weight.data.fill_(0.)
    self.times_bias.weight.data.fill_(0.)


  # Define forward propagation
  def forward(self, user, times):
    # print(self.user_factors(user).shape)
    # print(self.times_factors(times).shape)
    pred = self.user_factors(user) * self.times_factors(times)
    pred = pred.sum(1, keepdim=False)
    pred += self.user_bias(user).squeeze() + self.times_bias(times).squeeze()
    return pred

# Define Loss function excluding missing values
def loss_func(predicted, actual, mask, tau):
    # print(predicted.shape, actual.shape, mask.shape)
    invert_mask = ~mask
    # print(predicted[invert_mask].shape, actual[invert_mask].shape)

    ### MSE LOSS
    mseFUNC = nn.MSELoss();
    return mseFUNC(predicted[invert_mask].view(-1), actual[invert_mask].view(-1))


    loss = 0
    invert_mask = invert_mask.reshape(-1)
    count = 0
    for i in range(predicted.shape[0]):
      if(invert_mask[i]):
        count += 1
        difference = actual[i] - predicted[i]
        loss += tau * difference ** 2 if difference >= 0 else (1-tau) * difference**2
    # return loss(predicted[invert_mask].view(-1), actual[invert_mask].view(-1))
    loss /= count
    return loss

# Define parameters in our case
number_users, number_times = train_data.shape
rank = 6
model = LRModel(number_users, number_times, rank)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Epochs and CheckpointPath
number_epochs = 300

global_best_loss = float('inf')
best_epoch = 0

print(val_mask.shape)

tensor([[ 0.4797,  0.2491,  0.2675,  ..., -1.4358,     nan, -0.6271],
        [    nan,     nan,  0.7744,  ...,  0.7552, -2.4682,  1.2515],
        [-0.2632,  1.3360,     nan,  ...,     nan, -2.4719, -2.0264],
        ...,
        [    nan,     nan,  0.5399,  ...,  0.7345, -0.0150,  1.0831],
        [ 0.0783,  0.6225,     nan,  ...,  0.1428,  0.0469, -0.9662],
        [ 1.1254, -0.1971, -0.9016,  ...,  1.4889,  0.2518,     nan]])
[[False False False ... False  True False]
 [ True  True False ... False False False]
 [False False  True ...  True False False]
 ...
 [ True  True False ... False False False]
 [False False  True ... False False False]
 [False False False ... False False  True]]
(10, 40)


In [87]:
# Training
for epoch in range(number_epochs):
  user_indices = torch.arange(number_users).repeat_interleave(number_times)
  time_indices = torch.arange(number_times).repeat(number_users)
  output = model(user_indices, time_indices)

  # Calculate loss
  train_tensor_flat = train_tensor.view(-1)
  train_mask_flat = train_mask.reshape(-1)
  training_loss = loss_func(output, train_tensor_flat, train_mask_flat, 0.5)
  if(epoch % 10 == 0):
    print(f"Epoch {epoch} loss is : ", training_loss)

  # Backward
  optimizer.zero_grad()
  training_loss.backward()
  optimizer.step()

  with torch.no_grad():
    validation_num_user, validation_num_times = val_tensor.shape
    validation_user_indices = torch.arange(validation_num_user).repeat_interleave(validation_num_times)
    validation_time_indices = torch.arange(validation_num_times).repeat(validation_num_user)
    validation_output = model(validation_user_indices, validation_time_indices)
    validation_loss = loss_func(validation_output, val_tensor.view(-1), val_mask.reshape(-1), 0.5)
    # print("Validation Loss", validation_loss)

  # print(f"Epoch [{epoch + 1}/{number_epochs}]: Training Loss: {training_loss.item()}; Validation Loss: {validation_loss.item()}")

  if validation_loss < global_best_loss:
      global_best_loss = validation_loss
      best_epoch = epoch
      # torch.save({"Epoch": epoch, "Model_state_dict": model.state_dict(), "Optimizer_state_dict": optimizer.state_dict(), "Loss": validation_loss},
      #            f"/content/drive/MyDrive/low-rank-expectile/checkpoints/model_checkpoint_epoch{epoch}.pt")

print("Best Validation Epoch: ", best_epoch + 1)
print("Best Validation Loss: ", global_best_loss)



Epoch 0 loss is :  tensor(0.1300, grad_fn=<MseLossBackward0>)
Epoch 10 loss is :  tensor(0.1300, grad_fn=<MseLossBackward0>)
Epoch 20 loss is :  tensor(0.1300, grad_fn=<MseLossBackward0>)
Epoch 30 loss is :  tensor(0.1299, grad_fn=<MseLossBackward0>)
Epoch 40 loss is :  tensor(0.1299, grad_fn=<MseLossBackward0>)
Epoch 50 loss is :  tensor(0.1299, grad_fn=<MseLossBackward0>)
Epoch 60 loss is :  tensor(0.1299, grad_fn=<MseLossBackward0>)
Epoch 70 loss is :  tensor(0.1299, grad_fn=<MseLossBackward0>)
Epoch 80 loss is :  tensor(0.1298, grad_fn=<MseLossBackward0>)
Epoch 90 loss is :  tensor(0.1298, grad_fn=<MseLossBackward0>)
Epoch 100 loss is :  tensor(0.1298, grad_fn=<MseLossBackward0>)
Epoch 110 loss is :  tensor(0.1298, grad_fn=<MseLossBackward0>)
Epoch 120 loss is :  tensor(0.1298, grad_fn=<MseLossBackward0>)
Epoch 130 loss is :  tensor(0.1297, grad_fn=<MseLossBackward0>)
Epoch 140 loss is :  tensor(0.1297, grad_fn=<MseLossBackward0>)
Epoch 150 loss is :  tensor(0.1297, grad_fn=<MseLos